In [1]:
from typing import Any
import syft as sy

alice = sy.VirtualMachine(name="alice")
alice_client = alice.get_root_client()
remote_python = alice_client.syft.lib.python

In [2]:
remote_python

Module:
	.collections -> Module:
		.OrderedDict -> <syft.ast.klass.Class object at 0x7f8c378b6280>

	.Bool -> <syft.ast.klass.Class object at 0x7f8c37899e20>
	.Complex -> <syft.ast.klass.Class object at 0x7f8c37899e80>
	.Dict -> <syft.ast.klass.Class object at 0x7f8c37899ee0>
	.Float -> <syft.ast.klass.Class object at 0x7f8c37899f40>
	.Int -> <syft.ast.klass.Class object at 0x7f8c3788c700>
	.List -> <syft.ast.klass.Class object at 0x7f8c3788c940>
	.Slice -> <syft.ast.klass.Class object at 0x7f8c3788c460>
	.Range -> <syft.ast.klass.Class object at 0x7f8c378f89a0>
	.String -> <syft.ast.klass.Class object at 0x7f8c378f85e0>
	._SyNone -> <syft.ast.klass.Class object at 0x7f8c378b6040>
	.PyPrimitive -> <syft.ast.klass.Class object at 0x7f8c378b60a0>
	.Any -> <syft.ast.klass.Class object at 0x7f8c378b6100>
	.Tuple -> <syft.ast.klass.Class object at 0x7f8c378b6160>
	.Iterator -> <syft.ast.klass.Class object at 0x7f8c378b61c0>
	.Set -> <syft.ast.klass.Class object at 0x7f8c378b6220>

In [3]:
def get_permission(obj: Any) -> None:
    remote_obj = alice.store[obj.id_at_location]
    remote_obj.read_permissions[alice_client.verify_key] = obj.id_at_location

In [8]:
x_data = list(range(13))
y_data = [1, 3, 2, 5, 7, 8, 8, 9, 10, 12, 12, 13, 15]
zip(x_data, y_data)
x_data_ptr = sy.lib.python.List(x_data).send(alice_client)
y_data_ptr = sy.lib.python.List(y_data).send(alice_client)
print(f'x_data: {x_data} \n x_data_ptr: {x_data_ptr}')
print(f'y_data: {y_data} \n y_data_ptr: {y_data_ptr}')

x_data: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12] 
 x_data_ptr: <syft.proxy.syft.lib.python.ListPointer object at 0x7f8c3592c730>
y_data: [1, 3, 2, 5, 7, 8, 8, 9, 10, 12, 12, 13, 15] 
 y_data_ptr: <syft.proxy.syft.lib.python.ListPointer object at 0x7f8c35bd09a0>


In [12]:
n_ptr = x_data_ptr.__len__()
n_ptr

13

In [14]:
x_mean_ptr = remote_python.sum(x_data_ptr) / n_ptr
y_mean_ptr = sum(y_data_ptr) / n_ptr
x_mean_ptr


AttributeError: 'Module' object has no attribute 'sum'

In [ ]:
n_ptr = x_data_ptr.__len__()

x_mean_ptr = sum(x_data_ptr) / n_ptr
y_mean_ptr = sum(y_data_ptr) / n_ptr

crossdev_list_ptr = remote_python.List()
dev_list_ptr = remote_python.List()

for x_ptr, y_ptr in zip(x_data_ptr, y_data_ptr):
    crossdev_list_ptr.append(x_ptr * y_ptr)
    dev_list_ptr.append(x_ptr * x_ptr)


cross_dev_ptr = sum(crossdev_list_ptr) - n_ptr * x_mean_ptr * y_mean_ptr
dev_ptr = sum(dev_list_ptr) - n_ptr * x_mean_ptr * x_mean_ptr

b_1_ptr = cross_dev_ptr / dev_ptr
b_0_ptr = y_mean_ptr - b_1_ptr * x_mean_ptr

get_permission(b_1_ptr)
get_permission(b_0_ptr)

b_1 = b_1_ptr.get()
b_0 = b_0_ptr.get()

print(b_1)
print(b_0)
